=== GENERAL TODO'S:
                                       -|=======|=======|+
* Data read                                     ?
    * ~~Static~~                                        [done]
    * ~~Refrred to dicom fields~~                       [done]
    * Dynamic                                           [done]
        * interpolable                                  [done]
            * optimalization            [optional]
            * bicubic                   [optional]        
* Patchification system                         ?
    * ~~Static~~                                        [done]
    * Update-able                               [wip]
* Vascular segmentation                         [wip]
    * Linear kernel application                 [wip]
    * Canny                                             [done]
    * Canny -> Hull                                     [done]
    * Hull -> Mask                                      [done]
    * Mask -> Skel                                      [done]
    * Skel -> Segm                              [wip]
    + CNN?                              [reluctant]
* Vascular positioning                          ?
    * Vascular skel model               [concept]
    * Vascular skel discrimination      [concept]
    * Characteristic points estim       [concept]
* Visualisation
    * ~~Mango-like~~                                    [done]
        * Efficient                     [concept]
    * Rotable                           [concept]

python3 -m pip install cupy-cuda110


tdsp = r"../AngioCT tt szyjnych/4/AQWS01_10.7.112.230_20220525111102552/1.3.12.2.1107.5.1.4.105397.30000021102806505454700023291/1.3.12.2.1107.5.1.4.105397.30000021102806505454700023292.dcm"

In [1]:
## IMPORTS
import pydicom
import matplotlib.pyplot as plt
import glob
# import re
# import os
# import patchify
from tqdm import tqdm
import cv2
import datetime

import numpy as np
import cupy as cp

na                  = np.asarray
xm = {
    np:             (np.asarray, np.asarray, np.asarray, np),
    cp:             (cp.asarray, cp.asnumpy, cp.asarray, cp),
}  # (da, ha, xa, xp)
xdarray2xp          = lambda xdarray: {np.ndarray: np, cp.ndarray: cp}[type(xdarray)]

AXALL, AXNEW = (slice(None),), (None,)

ceil = lambda x: int(x) + (int(x) != x)

In [2]:
## DATA READ - BASIC
tdsp = r"../AngioCT tt szyjnych/4/AQWS01_10.7.112.230_20220525111102552/1.3.12.2.1107.5.1.4.105397.30000021102806505454700023291/*"
ds_path_l = glob.glob(tdsp)

ds_l = [pydicom.dcmread(ds_path) for ds_path in ds_path_l]
dsa  = na([ds.pixel_array for ds in ds_l[::-1]])

# for ds in ds_l: print(ds.AcquisitionTime)

# https://pydicom.github.io/pydicom/stable/auto_examples/image_processing/reslice.html#sphx-glr-auto-examples-image-processing-reslice-py
ds = ds_l[2]
ds.dir

## Image plane
# (0018, 0050) Slice Thickness                     DS: '0.6'                                            # [mm]                  Nominal slice thickness.            # https://dicom.innolitics.com/ciods/ct-image/image-plane/00180050
# (0020, 1041) Slice Location                      DS: '-28.5'+                                         # [mm] cz               
# (0028, 0030) Pixel Spacing                       DS: [0.416015625, 0.416015625]                       # [mm] *s(x, y)         (intuitive),                        # https://dicom.innolitics.com/ciods/ct-image/image-plane/00280030        
# (0020, 0032) Image Position (Patient)            DS: [-98.2919921875, -270.2919921875, -28.5]         # [mm]                  first image upper left corner       # https://dicom.innolitics.com/ciods/ct-image/image-plane/00200032
# (0020, 0037) Image Orientation (Patient)         DS: [1, 0, 0, 
#                                                       0, 1, 0]                                        # s2c(x, y, z) *s(x, y) projection mx cosines               # https://dicom.innolitics.com/ciods/ct-image/image-plane/00200037

##
# (0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'AXIAL', 'CT_SOM5 SPI']
# (0018, 9313) Data Collection Center (Patient)    FD: [0.0, -164.0, -28.5]
# (0018, 9318) Reconstruction Target Center (Patie FD: [8.0, -164.0, -28.5]
# (0028, 1050) Window Center                       DS: [105, 250]
# (0028, 1051) Window Width                        DS: [765, 500]
# (0028, 1052) Rescale Intercept                   DS: '-1024.0'
# (0028, 1053) Rescale Slope                       DS: '1.0'

## Time
# (0008, 0032) Acquisition Time                    TM: '111515.611000'


<bound method Dataset.dir of Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 212
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: CT Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 2.16.840.1.113669.632.21.1635916553.279322404.391980493922060650
(0002, 0010) Transfer Syntax UID                 UI: Implicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 2.16.840.1.113669.632.21.1
(0002, 0013) Implementation Version Name         SH: 'AQNET430-370IB5'
(0002, 0016) Source Application Entity Title     AE: ''
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'AXIAL', 'CT_SOM5 SPI']
(0008, 0016) SOP Class UID                       UI: CT Image Storage
(0008, 0018) SOP Instance UID         

In [3]:
## CODE > PATCHIFY
na = np.asarray

def pad_patchify(
        input_array: np.ndarray,                                ##(*bdim, *pdim) 
        padded_patch_shape,                                     ##(             pdim)
        pad_shape,                                              ##({shu+shr},   pdim)
        input_crop_shape,                                       ##({shu+shr},   pdim)
        patch_excess_min,                                       ##(             pdim)
        pad_mode="edge",                                        # numpy.pad(mode=)
        tv = None,                                           # str
):
    pdim    = padded_patch_shape.__len__()                      # patch dimensions no
    bdim    = input_array.shape.__len__() - pdim                # batch dimensions no

    ## calc actual lower padding (not being covered by crop margin)
    pad_shl,  pad_shr   = na(pad_shape)                         # shl, shu, shr = shape [span] lower, upper, rearwards
    crop_shl, crop_shr  = na(input_crop_shape)        
    arr2opa_shl         = crop_shl - pad_shl                    # position of opa's i0 relative to array (consider it vec); opa = outer-padded array
    opa2arr_shl         = pad_shl - crop_shl                    # postition of array's i0 relative to opa
    arr_crop_shl        = np.maximum(0, arr2opa_shl)            # reduce input cropping if no enough crop to satisfy padding
    opa_pad_shl         = np.maximum(0, opa2arr_shl)            # pad needing artificial fulfilling as not being covered by cropped image

    ## calc how much patches will be there  
    array_sh            = na(input_array.shape[ bdim:])
    crarr_sh            = array_sh - (crop_shl + crop_shr)      # crarr = cropped_array
    pdp_sh              = na(padded_patch_shape)                # pdp = padded_patch
    upd_sh              = pdp_sh - (pad_shl+pad_shr)            # upd = unpadded[_patch]
    patch_step          = upd_sh                                # step between 0th elements of consecutive patches
    upd_ex_min_sh       = na(patch_excess_min)
    pgrid_sh, pex_rm_sh = np.divmod(crarr_sh, patch_step)
    pex_accept          = pex_rm_sh > upd_ex_min_sh
    pgrid_sh            = pgrid_sh + pex_accept

    ## calc opa upper padding
    opa_sh              = (pgrid_sh * upd_sh) + (pad_shl + pad_shr)
    arr2opa_shu         = arr2opa_shl + opa_sh 
    opa2arr_shu         = opa2arr_shl + array_sh
    arr_crop_shu        = np.minimum(array_sh, arr2opa_shu)
    opa_pad_shu         = np.minimum(opa_sh,   opa2arr_shu)
    opa_pad_shr         = opa_sh - opa_pad_shu

    ## quick checks
    assert np.all(upd_sh > 0)                                   # is patch containing anything beside patch
    assert np.all(upd_ex_min_sh <= upd_sh)                      # is patch_excess_min positive
    assert np.all(pgrid_sh > 0)                                 # is possible to create at least one patch (by input shape, patch shape, patch excess)

    ## opa = input crop & outer-pad
    batch_sh            = na(input_array.shape[:bdim ])
    batch_all           = (slice(None),)    *bdim
    batch_pad           = (0,)              *bdim
    input_crop          = tuple(slice(aclx, acux)   for aclx, acux in zip(arr_crop_shl, arr_crop_shu))
    opa                 = np.pad(
                                        input_array[batch_all + input_crop], 
                            pad_width=  na((
                                            batch_pad + tuple(opa_pad_shl), 
                                            batch_pad + tuple(opa_pad_shr))).T, 
                            mode=       pad_mode) # type: ignore
    
    ## patchify
    patch_array         = np.zeros(())
    pgrid_og            = np.ogrid[tuple(slice(pgsx) for pgsx in pgrid_sh)]
    patch_og            = np.ogrid[tuple(slice(pasx) for pasx in pdp_sh)]
    pg2pa_og            = tuple(
                                (pgogx * pstepx)[(slice(None),) *pdim   + (None,)           *pdim] 
                            +   paogx           [(None,)        *pdim   + (slice(None),)    *pdim] 
                            for     pgogx,      pstepx,     paogx 
                            in zip( pgrid_og,   patch_step, patch_og))
    
    for _ in tqdm((None,), desc=f"{tv} [PBX={pgrid_sh.prod()}]", disable=not tv):
        patch_array     = opa[batch_all + pg2pa_og]

    ## save info about actual shape for merging
    pgrid_shape         = tuple(pgrid_sh)
    pex_crop_shr        = (upd_sh - pex_rm_sh) * pex_accept
    array_shape         = (pgrid_sh * upd_sh) - pex_crop_shr

    return patch_array, pgrid_shape, array_shape

def pad_unpatchify(
        patch_array: np.ndarray,
        padded_patch_shape,
        pad_shape,
        array_shape,
        tv = None
):
    pdim                = padded_patch_shape.__len__()
    bdim                = patch_array.shape.__len__() - (pdim *2)

    ## calc pads
    pdp_sh              = na(padded_patch_shape)
    pad_shl,  pad_shr   = na(pad_shape)
    upd_sh              = pdp_sh - (pad_shl+pad_shr)
    patch_step          = upd_sh

    pad_shu             = pdp_sh - pad_shr
    pgrid_shape         = patch_array.shape[bdim:-pdim]
    pgrid_sh            = na(pgrid_shape)

    ## unpatchify
    batch_shape         = patch_array.shape[:bdim ]
    batch_all           = (slice(None),) *bdim
    unpad_crop          = (slice(None),) *pdim + tuple(slice(pshlx, pshux) for pshlx, pshux in zip(pad_shl, pad_shu))
    pgrid_og            = np.ogrid[tuple(slice(pgsx) for pgsx in pgrid_sh)]
    patch_og            = np.ogrid[tuple(slice(pasx) for pasx in upd_sh)]
    pg2pa_og            = tuple(
                                (pgogx * pstepx)[(slice(None),) *pdim   + (None,)           *pdim] 
                            +   paogx           [(None,)        *pdim   + (slice(None),)    *pdim] 
                            for     pgogx,      pstepx,     paogx 
                            in zip( pgrid_og,   patch_step, patch_og))

    ## merge
    output_array_ex_sh  = pgrid_sh * upd_sh
    output_array_ex     = np.zeros((*batch_shape, *output_array_ex_sh), dtype=patch_array.dtype)

    for _ in tqdm((None,), desc=f"{tv} [PBX={pgrid_sh.prod()}]", disable=not tv):
        output_array_ex[batch_all + pg2pa_og] = patch_array[batch_all + unpad_crop]

    ## discard excess
    output_array_sh     = array_shape
    output_array        = output_array_ex[(...,) + tuple(slice(osx) for osx in output_array_sh)]

    return output_array

def batchify(
        pbatch_iter,                                            ##(bdim*p[g]dim, pdim)
        patch_shape,                                            ##(pdim)
        umem_pool,                                                # bytes / type.nbytes
        umem_poly,                                               # (1, root(pdim, mlt(patch_sh)), root(pdim, mlt(patch_sh)**2), ..., ) | %todo
        tv=None,
        xp=np,
):
    ## batch_size calc
    pdim                = patch_shape.__len__()
    umem_poly           = na(umem_poly)

    if  umem_poly.shape.__len__() == 1:                     # convert linear polynomial to dimensional one
        umem_poly_lin       = umem_poly
        umem_poly_lin_l     = umem_poly_lin.__len__()
        umem_poly_shx       = int(np.ceil((umem_poly_lin_l-1)/pdim) + 1)
        umem_poly_lin_lp    = (umem_poly_shx * pdim) - pdim + 1
        umem_poly_lin       = np.pad(umem_poly_lin, 
                                (0, umem_poly_lin_lp - umem_poly_lin_l), 
                                "constant", constant_values=0) # type: ignore
        umem_poly_map       = np.mgrid[(slice(umem_poly_shx),) *pdim].sum(axis=0)
        umem_poly           = umem_poly_lin[umem_poly_map]
        patch_shape         = (np.prod(patch_shape) **(1/pdim),) *pdim
    assert  umem_poly.shape.__len__() ==  pdim

    umem_deters         = np.multiply.reduce(na(tuple(
                            np.power(   pshx,           pogx) 
                            for         pshx,           pogx 
                            in zip(     patch_shape,    np.ogrid[tuple(slice(ups) for ups in umem_poly.shape)])),
                            dtype=object))
    umem_estim          = (umem_poly * umem_deters).sum()
    batch_size          = int(umem_pool // umem_estim)
    if not batch_size > 0:
        print(umem_pool/umem_estim)
    assert batch_size > 0

    ## batchify iterations
    # iters_nd            = xp.fromiter(pbatch_iter, dtype=int)
    iters_nd            = xp.asarray(list(pbatch_iter))
    iters_l             = iters_nd.__len__()
    iters_gen           = [((bind.__len__(),), tuple(bind.T))
                            for bix in range(0, iters_l, batch_size)
                            for bind in (iters_nd[bix:(bix+batch_size)],)]
    iters_tq            = tqdm(iters_gen, desc=f"{tv} [PBX={batch_size}/{iters_l}]", disable=not tv)
    return iters_tq
    

In [4]:
## DEFINE > LINEAR KERNELS

na = np.asarray

zero_kenrel = na([
    [[ 0,  0,  0], [ 0,  0,  0], [ 0,  0,  0]],
    [[ 0,  0,  0], [ 0,  0,  0], [ 0,  0,  0]],
    [[ 0,  0,  0], [ 0,  0,  0], [ 0,  0,  0]],

])

# X- Z| Y\
sobel_kernel3_gen = na([
    [[-1, -2, -1], [-2, -4, -2], [-1, -2, -1]],
    [[ 0,  0,  0], [ 0,  0,  0], [ 0,  0,  0]],
    [[+1, +2, +1], [+2, +4, +2], [+1, +2, +1]],
])
sobel_kernel3s = (
    sobel_kernel3_gen.swapaxes(0, 2), #z
    sobel_kernel3_gen.swapaxes(0, 1), #y
    sobel_kernel3_gen, #x
)[::-1] # (iAxis<kernel>, kZ, kY, kX)

sharpen_kernel3_gen = na([
    # [[ 0,  0,  0], [ 0,  0,  0], [ 0,  0,  0]],
    [[ 0, -1,  0], [-1,  5, -1], [ 0, -1,  0]],
    # [[ 0,  0,  0], [ 0,  0,  0], [ 0,  0,  0]],
])
sharpen_kernel3s = (
    sharpen_kernel3_gen,
    sharpen_kernel3_gen.swapaxes(0, 1),
    sharpen_kernel3_gen.swapaxes(0, 2),
)

sharpen_kernel3d3 = na([
    [[ 0,  0,  0], [ 0, -1,  0], [ 0,  0,  0]],
    [[ 0, -1,  0], [-1,  7, -1], [ 0, -1,  0]],
    [[ 0,  0,  0], [ 0, -1,  0], [ 0,  0,  0]],
])

In [5]:
## CODE > KERNEL APPLY
import cupy as cp
xd, xh, xa, xp          = xm[cp]  

def kernel_apply(patch_pd_shape_in, kernel, kernel_center=None):
    kernel_shape        = kernel.shape
    kernel_center       = kernel_center if kernel_center is not None else tuple((kshx-1)//2 for kshx in kernel_shape)
    assert patch_pd_shape_in.__len__() == kernel_shape.__len__()
    assert all(0 <= kcxx and kcxx < kshx for kcxx, kshx in zip(kernel_center, kernel_shape))

    pad_shape_io        = (
        tuple(kcx           for _,   kcx    in zip(kernel_shape, kernel_center)), 
        tuple((ksx-1)-kcx   for ksx, kcx    in zip(kernel_shape, kernel_center)))

    if (pad_shape_in:=None) is not None: 
        raise NotImplementedError
    else:
        kernel_adjust   = (0,) *kernel_shape.__len__()
        pad_shape_op    = ((0,) *kernel_shape.__len__(),)*2

    patch_pd_shape_op      = tuple(
        patch_shx - (pad_shxl + pad_shul)                      
            for patch_shx, pad_shxl, pad_shul in zip(patch_pd_shape_in, *pad_shape_io))    # unpadded preferably
        
    ixt_gen = tuple((
        kernel_ixt,                                                                                                             # kernel_mask
        tuple(slice(kix + kadjx, asx + kix + kadjx) for kix, asx, kadjx in zip(kernel_ixt, patch_pd_shape_op, kernel_adjust)),  # patch_mask
        ) for kernel_ixt in xp.ndindex(kernel_shape) if kernel[kernel_ixt] != 0
    )
    
    return patch_pd_shape_op, pad_shape_op, ixt_gen

def _kernel_apply2_iter(pgrid_shape, dtype, xp, ixt_gen, patch_pd_shape_op):
    patch_op = xp.zeros((*pgrid_shape, *patch_pd_shape_op), dtype=dtype)
    for ixt_gen_op in ixt_gen: yield (*ixt_gen_op, patch_op)

def kernel_apply2(patch_pd_shape_in, kernel, kernel_center=None, dtype=None):
    patch_pd_shape_op, pad_shape_op, ixt_gen = kernel_apply(patch_pd_shape_in, kernel, kernel_center)
    return patch_pd_shape_op, pad_shape_op, lambda pgrid_shape=(): _kernel_apply2_iter(pgrid_shape, dtype, xdarray2xp(kernel), ixt_gen, patch_pd_shape_op)

# dsa                 = dsa[tuple(slice(psx) for psx in patch_shape)]  # test# i := int(1e3) and dsa := np.zeros(i**3).reshape((i,)*3) # test

proc_dtype              = np.float16
dsi                     = dsa.astype(proc_dtype)#[200:300, 200:400, 100:300]
kernel                  = sobel_kernel3s[2].astype(proc_dtype)

kbim                    = 3 # kernel batch dimension
pdim                    = 3
patch_pdshin            = (16, )    *pdim               # patch_padded_shape_in
pad_shape_in            = ((1, )    *pdim,) *2

kernel_sobel_d          = xd(sobel_kernel3s)
kernel_sharp_d          = xd(sharpen_kernel3d3)

dspdin, pgrid_shape, amerge_shape   \
                        = pad_patchify(dsi, patch_pdshin, pad_shape_in, ((0,)*3,)*2, (0,)*3, tv="patchify")

_osh                    = patch_pdshin
_osh, _psh, iter_sobel  = kernel_apply2(_osh, kernel_sobel_d[0], (1,)*pdim, xp.float32)
patch_pdshop            = _osh
pad_shape_op            = _psh
dspaop_amp_pl              = np.zeros((3, *pgrid_shape, *patch_pdshop), dtype=proc_dtype)  # dtype=dspain.dtype) # % todo: parametrize when input data signed
dspaop_arc_pl              = np.zeros((3, *pgrid_shape, *patch_pdshop), dtype=proc_dtype)  # dtype=dspain.dtype) # % todo: parametrize when input data signed

for bl, bix             in batchify(np.ndindex(pgrid_shape), patch_pdshin, 
                            2**31 //2, (*(0,)*pdim, 120), tv="linear kernel batchified seqencing"):
    patch_in_d          = xd(dspdin[bix])

    for kernel_ixt, patch_sxt, op_sobel in iter_sobel((*bl, kbim)): 
        op_sobel[:] +=  patch_in_d[(*AXALL, *AXNEW, *patch_sxt)]   * kernel_sobel_d[(*AXALL, *kernel_ixt, *AXNEW*3)]
    patch_op_amp     = xp.sqrt(((op_sobel[:, [[1,2], [0,2], [0,1]]]) **2).sum(axis=2)).astype(proc_dtype) # ZYX (as normals)
    patch_op_arc     = xp.arctan2(op_sobel[:, [2,0,1]], op_sobel[:, [1,2,0]]).astype(proc_dtype) # ZYX

    dspaop_amp_pl[(*AXALL, *bix,)] = xh(patch_op_amp.swapaxes(1, 0))
    dspaop_arc_pl[(*AXALL, *bix,)] = xh(patch_op_arc.swapaxes(1, 0))


dsa_amp_pl          = pad_unpatchify(dspaop_amp_pl, patch_pdshop, pad_shape_op, amerge_shape, tv="unpatchify") #= aom[:, 0]
dsa_arc_pl          = pad_unpatchify(dspaop_arc_pl, patch_pdshop, pad_shape_op, amerge_shape, tv="unpatchify") #= aom[:, 1]

patchify [PBX=56129]: 100%|██████████| 1/1 [00:01<00:00,  1.65s/it]
linear kernel batchified seqencing [PBX=2184/56129]: 100%|██████████| 26/26 [00:08<00:00,  3.06it/s]
unpatchify [PBX=56129]: 100%|██████████| 1/1 [00:02<00:00,  3.00s/it]


In [6]:
## CODE > CIRCSEGM > INIT
index_dtype = xp.int16
proc_dtype  = xp.float32
radii_dtype = xp.float32
score_dtype = xp.float32
xd, xh, xa, xp = xm[cp]      
ndim=3       

mmclamm = 750
## V mm
cc_mask_stddev = 4
cc_mask_step = 2
cc_mask_threshold = .5
cc_score_cutoff = .5
cc_radii_cutoff = 4.1
cc_range = (2, 16, cc_mask_step)

ccrrl = list([1, 2]) + list(range(*cc_range))
ccrrl = list(range(*cc_range))
mm_clamp = (mmclamm, mmclamm+100)
ccrrl = sorted(ccrrl)
assert any(crx < cc_radii_cutoff for crx in ccrrl)

pad_cc_radius = ceil(max(ccrrl) + (cc_mask_stddev*2))
pad_ms_radius = ceil(max(ccrrl)) #//2

# sh_incircsegm = (20, 512, 512, )
sh_incircsegm = dsi.shape
innsh = 128

nd_2d2z = (*AXNEW, *AXALL, *AXALL)
nd_rollaxes = tuple(tuple((idim+jdim)%ndim for idim in range(ndim)) for jdim in range(ndim))
nd_z2i = lambda z2i, idim: xdarray2xp(z2i).moveaxis(z2i, nd_rollaxes[0], nd_rollaxes[idim])

if True:
    sh_padded_nd    = tuple(tuple((shx+pad_cc_radius*2) if shi!=idim else innsh for shi, shx in enumerate(sh_incircsegm)) for idim in range(ndim))
    sh_unpadd_nd    = tuple(tuple(na(sh_padded_nd[idim]) - (np.arange(ndim)   != idim)*2*pad_cc_radius) for idim in range(ndim))
    sh_mspadd_nd    = tuple(tuple(na(sh_unpadd_nd[idim]) + (np.arange(ndim)   != idim)*2*pad_ms_radius) for idim in range(ndim))
    sh_inppad_nd    = tuple((tuple((0 if idim==jdim else pad_cc_radius) for jdim in range(ndim)),)*2    for idim in range(ndim))
    sh_mspadd_ms    = tuple(tuple((0,)*2 if idim==jdim else (pad_ms_radius,)*2 for jdim in range(ndim)) for idim in range(ndim))
    # crop_ms     = tuple(tuple(slice(pad_cc_radius, -pad_cc_radius)      if idim!=jdim else slice(None)                          for jdim in range(ndim)) for idim in range(ndim))
    pad_ms_mxa = xp.mgrid[(slice(-pad_ms_radius, pad_ms_radius+1),)*2].astype(index_dtype)
    pad_ms_amp = xp.sqrt((pad_ms_mxa.astype(radii_dtype)**2).sum(axis=0))
    pad_ms_mask = (pad_ms_amp <= pad_ms_radius) * (pad_ms_amp > 0)
    iter_adjmax_mask_nd = tuple(tuple(
            (nd_z2i(pad_ms_amp[nd_2d2z], idim)[kernel_ixt], patch_sxt) 
                for kernel_ixt, patch_sxt in kernel_apply(sh_mspadd_nd[idim], nd_z2i(pad_ms_mask[nd_2d2z], idim))[-1])
        for idim in range(ndim))

    pad_cc_mxa = xp.mgrid[(slice(-pad_cc_radius, pad_cc_radius+1),)*2].astype(index_dtype)
    pad_cc_amp = xp.sqrt((pad_cc_mxa.astype(radii_dtype)**2).sum(axis=0))
    # pad_cc_arc_nd = tuple(xp.arctan2(pad_cc_mxa[nd_rollaxes[idim][1]], pad_cc_mxa[nd_rollaxes[idim][0]]) for idim in range(ndim))
    pad_cc_arc = xp.arctan2(pad_cc_mxa[1], pad_cc_mxa[0])

circ_gen_zyx = []
for idim in range(ndim):
    circ_gen_i = []
    for ixcr, ccrr in enumerate(ccrrl):
        cc_mask_stddev = cc_mask_stddev 
        ccr_rim = ccrr
        cc_divs = ccr_rim*2

        cc_mask = xp.exp(-((pad_cc_amp-ccr_rim)/cc_mask_stddev)**2)
        cc_mask[cc_mask < cc_mask_threshold] = 0
        cc_mask /= (cc_divs)

        cc_mask_nd = nd_z2i(cc_mask[nd_2d2z], idim)

        cc_arc_discr = (2*xp.pi)/cc_divs
        cc_mask_arc_discr = xp.floor((pad_cc_arc/cc_arc_discr))
        cc_mad_nd = nd_z2i(cc_mask_arc_discr[nd_2d2z], idim)
        # cc_mask_arc = ((cc_mask_arc_discr+.5)*cc_arc_discr)

        iter_ccmask_rad = tuple((
                (cc_discr+.5)*cc_arc_discr,
                tuple((
                    tuple(kernel_ixt),
                    tuple(slice(kix, asx + kix) for kix, asx in zip(kernel_ixt, sh_unpadd_nd[idim])),
                ) for kernel_ixt in xa(xp.where((cc_mask_nd!=0)*(cc_mad_nd==cc_discr))).astype(index_dtype).T))
            for cc_discr in range(-cc_divs//2, cc_divs//2+1))
        
        circ_gen_i.append((cc_mask_nd, ccr_rim, iter_ccmask_rad))
    circ_gen_zyx.append(circ_gen_i)
print("INIT_DONE")
# sh_padded_nd, sh_unpadd_nd, sh_mspadd_nd, sh_inppad_nd, sh_mspadd_ms

INIT_DONE


In [7]:
## CODE > CIRCULAR SEGMENTATION
xd, xh, xa, xp = xm[cp]   
ndim = 3
ndim = 0 ## DISABLE
onlyz = True

image_c = dsa_amp_pl
image_c = np.clip((image_c-mm_clamp[0])/(mm_clamp[1]-mm_clamp[0]), 0, 1)
image_a = dsa_arc_pl

if ndim > 0:
    ds_circsegmout = np.zeros((ndim,) + dsa_amp_pl[0].shape, dtype=radii_dtype)
for idim in range(ndim):
    if onlyz and idim!=0: break

    image_amp, pgrid_shape, amerge_shape = pad_patchify(image_c, sh_padded_nd[idim], sh_inppad_nd[idim], ((0,)*3,)*2, (0,)*3, tv=f"patchify amp {idim}/{ndim}")
    image_arc, pgrid_shape, amerge_shape = pad_patchify(image_a, sh_padded_nd[idim], sh_inppad_nd[idim], ((0,)*3,)*2, (0,)*3, tv=f"patchify arc {idim}/{ndim}")
    ds_circsegmout_i = np.zeros(pgrid_shape + sh_unpadd_nd[idim], dtype=radii_dtype)

    for bl, bix         in batchify(np.ndindex(pgrid_shape), patch_pdshin, 1, (1,0,0,0,0), tv=f"circular segmentation batchified sequencing {idim}/{ndim}"):
        image_ampb  = xd(image_amp[(idim,) + bix][0])
        image_arcb  = xd(image_arc[(idim,) + bix][0])

        circ_score = xp.zeros(sh_unpadd_nd[idim], dtype=score_dtype)
        circ_radii = xp.zeros(sh_unpadd_nd[idim], dtype=radii_dtype)

        for (cc_mask_d, ccr_rim, iter_ccmask_rad,) in circ_gen_zyx[idim]:
            pdop_ccmask_rad = xp.zeros(sh_unpadd_nd[idim], dtype=score_dtype)
            for kernel_arc, iter_ccmask in iter_ccmask_rad:
                pdop_ccmask_ri = xp.zeros(sh_unpadd_nd[idim], dtype=score_dtype)
                image_arcr = xp.cos(image_arcb-kernel_arc)**8
                for kernel_ixt, patch_sxt, in iter_ccmask: 
                    pccm = image_ampb[patch_sxt] * cc_mask_d[kernel_ixt] * image_arcr[patch_sxt]
                    pdop_ccmask_ri = xp.maximum(pdop_ccmask_ri, pccm)
                pdop_ccmask_rad += pdop_ccmask_ri

            circ_score_recalc = pdop_ccmask_rad
            ccovsc_bxa = (circ_score < circ_score_recalc)
            circ_radii = np.where(ccovsc_bxa, ccr_rim,              circ_radii)
            circ_score = np.where(ccovsc_bxa, circ_score_recalc,    circ_score)

        circ_score_pd = xp.pad(circ_score, sh_mspadd_ms[idim], constant_values=0)
        circ_radii_pd = xp.pad(circ_radii, sh_mspadd_ms[idim], constant_values=0)

        ccscore_redux =  (cc_radii_cutoff <= circ_radii) * (cc_score_cutoff <= circ_score)
        ccscore_redux_pd = xp.pad(ccscore_redux, sh_mspadd_ms[idim], constant_values=0)

        ms_radii = circ_radii * ccscore_redux
        for kernel_rad, patch_sxt in iter_adjmax_mask_nd[idim]: 
            ms_radii += (ccscore_redux_pd[patch_sxt] 
                * (circ_radii_pd[patch_sxt] + kernel_rad > circ_radii) 
                * (1/(2*xp.pi*kernel_rad))
            )
        ms_radii_pd = xp.pad(ms_radii, sh_mspadd_ms[idim], constant_values=0)

        circ_filteredout = ms_radii * ccscore_redux
        for kernel_rad, patch_sxt in iter_adjmax_mask_nd[idim]: 
            circ_filteredout *= (
                (ms_radii_pd[patch_sxt] <= ms_radii) # ms circ bigger
                + (circ_radii_pd[patch_sxt] <= kernel_rad) #ms circ or lesser and out of range
            )
        ds_circsegmout_i[bix] = xh(circ_filteredout) #[crop_ms[idim]]
    ds_circsegmout[idim] = pad_unpatchify(ds_circsegmout_i, sh_unpadd_nd[idim], ((0,)*3,)*2, amerge_shape, tv=f"output unpatchify {idim}/{ndim}")

try:
    del image_c, image_a, 
    del image_amp, image_arc, 
    del image_ampb, image_arcb, 
    del pdop_ccmask_ri, pccm, pdop_ccmask_rad, 
    del circ_score, circ_score_pd, 
    del circ_score_recalc, ccovsc_bxa
    del circ_radii, circ_radii_pd, 
    del ccscore_redux, ccscore_redux_pd
    del ms_radii, ms_radii_pd, circ_filteredout
    del ds_circsegmout_i,
except: pass

In [8]:
import gzip
# with gzip.open("./circsegmout2.gz", "wb") as f:
#     np.save(f, xh(ds_circsegmout), allow_pickle=True)

with gzip.open("./circsegmout2.gz", "rb") as f:
    ds_circsegmout = np.load(f, allow_pickle=True)

In [9]:
## VERTSEGM > CLASSIFY SEGMENTS
ndim = 3
idim = 0
# dscs_db = xd(ds_circsegmout[idim, :, 200:300, 200:300])
dscs_db = xd(ds_circsegmout[idim, :])

dscs_where = xp.where(dscs_db)
dscs_radii = dscs_db[dscs_where]
dscs_coord = xa(dscs_where).T

dscs_junct = xp.zeros(dscs_coord.shape[0], dtype=bool)
dscs_segme = xp.zeros(dscs_coord.shape[0], dtype=bool)

dscs_sjmap = xp.zeros(dscs_coord.shape[0], dtype=index_dtype)
dscs_segml = xp.zeros(dscs_coord.shape[0], dtype=index_dtype)
dscs_segmu = xp.zeros(dscs_coord.shape[0], dtype=index_dtype)

segm_origs_direct = xp.zeros((0, 2), dtype=index_dtype)

px, py = ((idim+1)%ndim,), ((idim+2)%ndim,)
radconnsq = lambda rm, rx: xp.minimum(rm, rx)**2 #(rm**2)+(rx**2) #xp.maximum(rm, rx)**2 #(rm+rx)**2#

dl_axa, dl_coord, dl_radii = xp.zeros((0,), dtype=index_dtype), xp.zeros((0, ndim), dtype=index_dtype), xp.zeros((0,), dtype=radii_dtype)[(*AXNEW,)].T
dm_axa, dm_coord, dm_radii = xp.zeros((0,), dtype=index_dtype), xp.zeros((0, ndim), dtype=index_dtype), xp.zeros((0,), dtype=radii_dtype)[(*AXNEW,)].T
for _, idx in enumerate(range(dscs_db.shape[idim]+1)):  # dl -> dm -> du; classify junctions and segment elements 
    du_axa      = xp.where(dscs_coord[:, idim]==idx)[0]
    du_coord    = dscs_coord[du_axa]
    du_radii    = dscs_radii[du_axa][(*AXNEW,)].T
    connl = ((dm_coord[:, px] - dl_coord[:, px].T)**2 + (dm_coord[:, py] - dl_coord[:, py].T)**2) <= radconnsq(dm_radii, dl_radii.T)
    connu = ((dm_coord[:, px] - du_coord[:, px].T)**2 + (dm_coord[:, py] - du_coord[:, py].T)**2) <= radconnsq(dm_radii, du_radii.T)
    connl_msum = connl[..., None].sum(axis=1).squeeze()
    connu_msum = connu[..., None].sum(axis=1).squeeze()

    pm_contsegm = ((connl_msum == 1) * (connu_msum == 1))
    dscs_segme[dm_axa[pm_contsegm]] = True

    pm_origin_l = ((connl_msum == 0) * (connu_msum > 0)) + ((connl_msum == 1) * (connu_msum > 1))
    pm_origin_u = ((connu_msum == 0) * (connl_msum > 0)) + ((connu_msum == 1) * (connl_msum > 1))
    pm_junct_ex = ((connl_msum > 2) + (connu_msum > 2))
    pm_junct_in = pm_origin_l + pm_origin_u + pm_junct_ex
    # pm_junct_in = ~pm_contsegm #
    dscs_junct[dm_axa[pm_junct_in]] = True

    dl_axa, dl_coord, dl_radii = dm_axa, dm_coord, dm_radii
    dm_axa, dm_coord, dm_radii = du_axa, du_coord, du_radii


dl_axa, dl_coord, dl_radii, dl_junct, dl_segme = xp.zeros((0,), dtype=index_dtype), xp.zeros((0, ndim), dtype=index_dtype), xp.zeros((0,), dtype=radii_dtype)[(*AXNEW,)].T, xp.zeros((0,), dtype=index_dtype)[(*AXNEW,)].T , xp.zeros((0,), dtype=index_dtype)[(*AXNEW,)].T 
dm_axa, dm_coord, dm_radii, dm_junct, dm_segme = xp.zeros((0,), dtype=index_dtype), xp.zeros((0, ndim), dtype=index_dtype), xp.zeros((0,), dtype=radii_dtype)[(*AXNEW,)].T, xp.zeros((0,), dtype=index_dtype)[(*AXNEW,)].T , xp.zeros((0,), dtype=index_dtype)[(*AXNEW,)].T 
for _, idx in enumerate(range(dscs_db.shape[idim]+1)):  # dl -> dm -> du; build segments
    du_axa      = xp.where((dscs_coord[:, idim]==idx))[0]
    du_coord    = dscs_coord[du_axa]
    du_radii    = dscs_radii[du_axa][(*AXNEW,)].T 
    du_junct    = dscs_junct[du_axa][(*AXNEW,)].T 
    du_segme    = dscs_segme[du_axa][(*AXNEW,)].T

    connl = ((dm_coord[:, px] - dl_coord[:, px].T)**2 + (dm_coord[:, py] - dl_coord[:, py].T)**2) <= radconnsq(dm_radii, dl_radii.T)
    connu = ((dm_coord[:, px] - du_coord[:, px].T)**2 + (dm_coord[:, py] - du_coord[:, py].T)**2) <= radconnsq(dm_radii, du_radii.T)

    sm_juncl_junct, sl_juncl_junct = xp.where(connl * dm_junct * dl_junct.T) 
    segm_origs_direct = xp.concatenate((segm_origs_direct, xp.vstack((dl_axa[sl_juncl_junct], dm_axa[sm_juncl_junct])).T), axis=0)

    sm_segml_junct, sl_segml_junct = xp.where(connl * dm_segme * dl_junct.T)  
    sm_segml_segme, sl_segml_segme = xp.where(connl * dm_segme * dl_segme.T)  
    sm_segmu_junct, su_segmu_junct = xp.where(connu * dm_segme * du_junct.T)

    # print(idx)
    # print(dm_axa[sm_segml_segme], dl_axa[sl_segml_segme])
    # print(dl_axa[sl_juncl_junct], dm_axa[sm_juncl_junct])
    # print(dscs_coord[dl_axa[sl_juncl_junct]], dscs_coord[dm_axa[sm_juncl_junct]])

    dscs_sjmap[dm_axa[sm_segml_junct]] = dl_axa[sl_segml_junct]
    dscs_sjmap[dm_axa[sm_segmu_junct]] = du_axa[su_segmu_junct]

    dscs_segml[dm_axa[sm_segml_junct]] = dm_axa[sm_segml_junct]             # if new segment started, mark first segment element as self
    dscs_segml[dm_axa[sm_segml_segme]] = dscs_segml[dl_axa[sl_segml_segme]] # if segment is continued, mark continuing element as first one
    segmu_map = xp.where((dscs_segml[:, None] == dscs_segml[dm_axa[sm_segmu_junct]][None, :]))
    dscs_segmu[segmu_map[0]] = dm_axa[sm_segmu_junct][segmu_map[1]]         # if segment is ending, mark all segment element as last one    

    dl_axa, dl_coord, dl_radii, dl_junct, dl_segme = dm_axa, dm_coord, dm_radii, dm_junct, dm_segme
    dm_axa, dm_coord, dm_radii, dm_junct, dm_segme = du_axa, du_coord, du_radii, du_junct, du_segme



In [11]:
# dscs_db.shape[idim]

# a = na([[0,1],[1,2], [1,2], [1,2],[2,3],[3,4]])
# b = na([[1,2], [2,3]])

# w= np.where(((a[:, None])==(b[None, :])).all(axis=-1))
# a[w[0]] =  b[w[1]]+10
# a

# a = na([0,1,1,2, 1,2, 1,2,2,3,3,4])
# b = na([1,2,3])

# w= np.where(((a[:, None])==(b[None, :])))
# a[w[0]] =  b[w[1]]+10
# a

# xh(dscs_segme).sum()
# b
for i in range(segm_origs_direct.__len__()): print(dscs_coord[segm_origs_direct[i]])

[[  0 176 113]
 [  1 177 113]]
[[  0 177 113]
 [  1 177 113]]
[[  0 209 203]
 [  1 211 204]]
[[  0 233 224]
 [  1 233 224]]
[[  0 234 224]
 [  1 233 224]]
[[  0 279 448]
 [  1 278 447]]
[[  0 279 448]
 [  1 278 448]]
[[  0 316 441]
 [  1 317 443]]
[[  0 317 463]
 [  1 319 462]]
[[  0 362 396]
 [  1 362 397]]
[[  0 487 218]
 [  1 488 219]]
[[  1 180 362]
 [  2 181 363]]
[[  1 180 362]
 [  2 181 364]]
[[  1 296  62]
 [  2 296  63]]
[[  1 347 406]
 [  2 349 405]]
[[  2 297 344]
 [  3 292 347]]
[[  2 349 405]
 [  3 349 399]]
[[  2 403 374]
 [  3 400 376]]
[[  2 492 275]
 [  3 489 278]]
[[  3 146 223]
 [  4 143 218]]
[[  3 146 223]
 [  4 146 235]]
[[  3 188 258]
 [  4 190 258]]
[[  3 208 265]
 [  4 208 267]]
[[  3 210 373]
 [  4 208 379]]
[[  3 210 373]
 [  4 209 373]]
[[  3 291 407]
 [  4 292 407]]
[[  3 305 409]
 [  4 312 408]]
[[  3 325  62]
 [  4 313  60]]
[[  3 492 231]
 [  4 492 232]]
[[  4 143 218]
 [  5 145 236]]
[[  4 146 235]
 [  5 145 236]]
[[  4 190 258]
 [  5 174 252]]
[[  4 19

In [13]:
## VERT SEGM > CONNECT JUNCTS
xd, xh, xa, xp = xm[cp]   

segm_origs = segm_origs_direct
segm_uniqs = segm_origs_direct

segm_bfl_direct = xp.zeros((0, 2), dtype=index_dtype)
segm_sco_direct = xp.zeros((0,),   dtype=index_dtype)

segm_build = segm_uniqs.reshape((-1, 1, 2))
while True:
    segm_furth = segm_uniqs[xp.isin(segm_uniqs[:, 0], segm_build[:, -1, 1])]
    if segm_furth.shape[0] == 0: break

    segm_bfi = xp.isin(segm_build[:, -1, 1], segm_furth[:, 0])
    # segm_bfl.extend(xh(segm_build))
    segm_bfl_direct = xp.concatenate((segm_bfl_direct, segm_build[:, [0, -1], [0, 1]]))

    segm_score = (segm_build[:, None]==segm_origs[None, :, None])[..., None].all(axis=-2).any(axis=-2).sum(axis=-2)[..., 0]
    segm_sco_direct = xp.concatenate((segm_sco_direct, segm_score))
    segm_build = segm_build[segm_bfi]

    wh_build, wh_furth = xp.where(segm_build[:, -1, 1][:, None] == segm_furth[:, 0][None, :])
    segm_build = xp.concatenate((segm_build[wh_build], segm_furth[wh_furth].reshape(-1, 1, 2)), axis=1)


In [14]:
## VERTSEGM > CONNECT SEGMENTS
xd, xh, xa, xp = xm[cp]   

segm_origs = xp.stack((
        dscs_sjmap[dscs_segml[dscs_segme]], 
        dscs_sjmap[dscs_segmu[dscs_segme]]
    ), axis=1)
segm_origs = segm_origs[~(segm_origs[:, 0] == segm_origs[:, 1])]

segm_uniqs = xd(np.unique(xh(segm_origs), axis=0))
segm_conns = xd(np.unique(xh(segm_bfl_direct), axis=0))
segm_conns = segm_conns[xp.isin(segm_conns[:, 0], segm_uniqs[:, 1]) * xp.isin(segm_conns[:, 1], segm_uniqs[:, 0])]
print(segm_uniqs.shape, segm_conns.shape)
segm_uniqs = xp.concatenate((segm_uniqs, segm_conns))
segm_uniqs = segm_uniqs[(segm_uniqs[:, 0] < segm_uniqs[:, 1])]
segm_count = (segm_origs[None, :] == segm_uniqs[:, None])[..., None].all(axis=-2).sum(axis=-2).squeeze()

# for i in range(segm_uniq.__len__()): 
#     if segm_cont[i] > 0: print(dscs_coord[segm_uniq[i][0]], dscs_coord[segm_uniq[i][1]], segm_cont[i])
segm_bfl = []
segm_sco = xp.zeros((0,), dtype=index_dtype)

segm_build = segm_uniqs[~xp.isin(segm_uniqs[:, 0], segm_uniqs[:, 1])].reshape((-1, 1, 2))
while True:
    segm_furth = segm_uniqs[xp.isin(segm_uniqs[:, 0], segm_build[:, -1, 1])]
    if segm_furth.shape[0] == 0: break

    segm_bfi = xp.isin(segm_build[:, -1, 1], segm_furth[:, 0])
    segm_build_fin = segm_build[~segm_bfi]
    segm_bfl.extend(xh(segm_build_fin))
    # print(segm_build_fin.shape, segm_furth.shape, segm_furth)

    segm_score = (segm_build_fin[:, None]==segm_origs[None, :, None])[..., None].all(axis=-2).any(axis=-2).sum(axis=-2)[..., 0]
    segm_sco = xp.concatenate((segm_sco, segm_score))
    segm_build = segm_build[segm_bfi]

    wh_build, wh_furth = xp.where(segm_build[:, -1, 1][:, None] == segm_furth[:, 0][None, :])
    segm_build = xp.concatenate((segm_build[wh_build], segm_furth[wh_furth].reshape(-1, 1, 2)), axis=1)

# print([type(x) for x in segm_bfl if type(x) is not cp.ndarray])
segm_bfl = np.asarray(segm_bfl, dtype=object)

(2205, 2) (154, 2)


In [15]:
# n = xp.where((ii == xa([[89, 8, 58], [266, 26, 56]])).all(axis=(-1, -2)))[0]
# for i in range(n.__len__()-1): print(dscs_coord[n[i]], (dscs_coord[n[i], 1]-dscs_coord[n[i+1], 1])**2+(dscs_coord[n[i], 2]-dscs_coord[n[i+1], 2])**2, dscs_radii[n[i]]**2)

In [43]:
# segm_origs.shape, , 
jcl = xp.unique(segm_origs[:, 0])
jcu = xp.unique(segm_origs[:, 1])
jcl_coord = dscs_coord[jcl]
jcu_coord = dscs_coord[jcu]

arc_tol = .25 * xp.pi
dst_tol = 30

jc_pols = jcl_coord[:, (idim,)] - jcu_coord[:, (idim,)].T
jc_rsq_ = (jcl_coord[:, px] - jcu_coord[:, px].T)**2 + (jcl_coord[:, py] - jcu_coord[:, py].T)**2
jc_rads = xp.sqrt(jc_rsq_)
jc_dsts = xp.sqrt(jc_rsq_+(jc_pols)**2)
jc_sins = xp.sin(xp.arctan2(jc_pols, jc_rads))

soff_l, soff_u = xp.where((jc_dsts <= dst_tol) * (jc_sins >= xp.cos(arc_tol)))
segm_origs_ffil = xp.stack((jcl[soff_l], jcu[soff_u]), axis=1)
segm_origs_ffil.shape
# juncts = dscs_coord[dscs_junct]

# j_rads = xp.sqrt(((juncts[:, px] - juncts[:, px].T)**2) + (juncts[:, py] - juncts[:, py].T)**2)
# j_pole = juncts[:, (idim)]
# juncts_sin = xp.arcsin(-(juncts[:])

(1566, 2)

In [74]:
print(segm_bfl.shape, segm_sco.shape)
for i in range(segm_sco.__len__()): print(dscs_coord[segm_bfl[i]], segm_sco[i])

ss = segm_bfl[xh(segm_sco >= 1)]
ss = np.concatenate([dscs_coord[ssx][[0, -1], [0, -1]][None] for ssx in ss])
ss.shape

(1822,) (1822,)
[[[  0  31 256]
  [  6  41 255]]] 5
[[[  0  31 256]
  [562 290 365]]] 2
[[[  0  31 256]
  [562 296 380]]] 383
[[[  0  62 254]
  [  6  41 255]]] 5
[[[  0 115 123]
  [ 60 116 128]]] 59
[[[  0 117 229]
  [ 33 126 221]]] 32
[[[  0 120 267]
  [  6 125 268]]] 5
[[[  0 149 150]
  [ 30 159 164]]] 29
[[[  0 161 340]
  [ 28 181 321]]] 27
[[[  0 186 231]
  [  6 226 216]]] 9
[[[  0 207 380]
  [  4 208 379]]] 3
[[[  0 218 241]
  [ 15 213 237]]] 14
[[[  0 233 206]
  [  5 231 210]]] 4
[[[  0 233 265]
  [  9 234 272]]] 8
[[[  0 251 392]
  [  7 248 388]]] 6
[[[  0 293 456]
  [ 21 278 442]]] 20
[[[  0 315  55]
  [  4 313  60]]] 3
[[[  0 330 412]
  [  5 327 400]]] 4
[[[  0 378  80]
  [  3 377  83]]] 2
[[[  0 382 384]
  [  6 375 388]]] 5
[[[  0 419 102]
  [  3 415 109]]] 2
[[[  0 429 117]
  [  4 426 113]]] 3
[[[  1 269 399]
  [  4 266 390]]] 2
[[[  1 319 462]
  [  5 306 456]]] 3
[[[  1 319 462]
  [ 41 344 460]]] 39
[[[  2 349 405]
  [  6 346 408]]] 3
[[[  3  58 229]
  [  6  60 227]]] 2
[[[

(1822, 2, 3)

In [75]:
xd, xh, xa, xp = xm[cp]

def drawz(shape, s12l, rad, val):
    oxt = xp.ogrid[tuple(slice(shx) for shx in shape)]
    out = xp.zeros(shape, proc_dtype)
    for s1, s2 in s12l:
        if s1[0] == s2[0]: continue
        if s1[0]  > s2[0]: s1, s2 = s2, s1
        s2[0] += 1
        vec = s2 - s1
        vec = vec / vec[0]
        zy = oxt[1] - ((oxt[0][s1[0]:s2[0]]-s1[0])*vec[1] + s1[1])
        zx = oxt[2] - ((oxt[0][s1[0]:s2[0]]-s1[0])*vec[2] + s1[2])
        zd = ((zy**2) + (zx**2) < rad**2) * val
        out[s1[0]:s2[0]] = xp.maximum(out[s1[0]:s2[0]], zd)
        del zy, zx, zd
    # del oxt[0], oxt[1], oxt[2]
    return out
    # test = drawz(dscs_db.shape, [[xa([100,100,100]), xa([500,500,500])]], 5, 1)

# print(ds_circsegmout.shape) #dsa.shape, xh(drawz(dsa.shape, xd(ss), 5, 2e3)).shape, xh((ds_circsegmout[0]>0)*4e3).shape
t = dsa + xh(drawz(dsa.shape, xd(ss), 10, 2e3)) + xh((ds_circsegmout[0]>0)*4e3)


In [46]:
t[tuple(xh(dscs_coord[dscs_segme].T))] += 16e3

In [35]:
dsd = xh((dsa/4096*255).astype(xp.uint8))
for i in range(dsd.shape[0]):
    si = dscs_coord[:, 0]==i
    cl = dscs_coord[si][:, [2,1]]
    rl = dscs_radii[si]
    for xy, r in zip(cl, rl):
        cv2.circle(dsd[i], xh(xy), int(xh(r)), 255, 1)


In [76]:
## VIS -> DCA
# from numba import jit, njit

# %matplotlib ipympl
%matplotlib qt
plt.ion()

view_dsa_l = [
    # dsa+(xh(ds_circsegmout[0]>0) * 1e4),
    # dsa+(xh(ds_circsegmout[1]>0) * 1e4),
    # dsa+(xh(ds_circsegmout[2]>0) * 1e4),
    # ds_circsegmout[0, 100:400, 100:400, 100:400]
    t,
    dsd,
]
view_center_axes_names = "ZYX"
view_slice_axes = ((1, 2), (0, 2), (0, 1))  # common for v_dsa
view_center_l = [np.asarray(dsax.shape) //2 for dsax in view_dsa_l]

figsize = 5
vfig_res = (view_dsa_l.__len__(), view_slice_axes.__len__())
vfig, vax_l = plt.subplots(
    *vfig_res,
    figsize=[figsize*vfr for vfr in vfig_res[::-1]])
if vax_l.shape.__len__() == 1:
    vax_l = vax_l[None, ...]

vai_l = np.zeros(vax_l.shape, dtype=list)
for vdsa_ix, vdsa in enumerate(view_dsa_l):
    for vsx_ix, vsxx in enumerate(view_slice_axes):
        vaxx = vax_l[vdsa_ix][vsx_ix]
        blank_im = np.zeros(
            (vdsa.shape[vsxx[0]], vdsa.shape[vsxx[1]]),
            dtype = vdsa.dtype)
        blank_im[0, 0] = vdsa.max()
        axis_image = vaxx.imshow(blank_im)#, cmap='gray')
        axis_center = vaxx.plot([0], [0], marker='+', c='r')
        axis_title = vaxx.set_title("")
        axis_ylabel = vaxx.set_ylabel("")
        
        vai_l[vdsa_ix][vsx_ix] = [
            axis_image,
            axis_center[0],
            axis_title,
            axis_ylabel,
        ]

# @jit(nopython=False)
def update_dsa_view(vdsa_ix):
    vai = vai_l[vdsa_ix]
    vsx = view_slice_axes
    vce = view_center_l[vdsa_ix]
    vdsa = view_dsa_l[vdsa_ix]

    for axex, axes in enumerate(vsx):
        indices = tuple((vce[axix] if axix not in axes else slice(None)) 
            for axix in range(vce.__len__()))
        sliced = vdsa[indices] # if axes[0] < axes[1] else vdsa[indices].T
        # print(vax[axex].imshow(sliced))
        vai[axex][0].set_data(sliced)

        slice_center = ([vce[axes[1]]], [vce[axes[0]]])
        vai[axex][1].set_data(*slice_center)

        indices_str = ", ".join(tuple(f"{axix_name}={vce[axix]}" if axix not in axes else f"{axix_name}:" 
            for axix, axix_name in enumerate(view_center_axes_names)))
        vai[axex][2].set_text(indices_str)

    vai[0][3].set_text(str(vce))
    vfig.canvas.draw()
    # vfig.canvas.flush_events()
    # plt.gcf().canvas.draw()

def update_center(vdsa_ix, vax_ix, xy_center):
    view_center = view_center_l[vdsa_ix]
    slice_axes_updated = view_slice_axes[vax_ix]
    view_center[slice_axes_updated[0]] = xy_center[1]
    view_center[slice_axes_updated[1]] = xy_center[0]

def mouse_callback(event):
    on_canvas = event.inaxes is not None
    xy = (int(event.xdata), int(event.ydata)) if on_canvas else (None, None)
    vdsa_ix, vax_ix = np.argwhere(vax_l == event.inaxes)[0] if on_canvas else (None, None)
    button, dc = (int(event.button), event.dblclick)
    # print(event)
    # print(on_canvas, xy, (vdsa_ix, vax_ix), (button, dc))

    if on_canvas:
        if button == 1:
            update_center(vdsa_ix, vax_ix, xy)
            update_dsa_view(vdsa_ix)
        elif button == 3:
            for vdsa_ix in range(view_dsa_l.__len__()):
                update_center(vdsa_ix, vax_ix, xy)
                update_dsa_view(vdsa_ix)

for vdsa_ix in range(view_dsa_l.__len__()):
    update_dsa_view(vdsa_ix)

cid = vfig.canvas.mpl_connect('button_press_event', mouse_callback)
vfig.tight_layout()
# vfig.canvas.draw()

In [70]:
## CODE > CIRCULAR SEGMENTATION > VIEW 2D
%matplotlib ipympl

proc_dtype  = xp.float16
radii_dtype = xp.float16
score_dtype = xp.float16
xd, xh, xa, xp              = xm[np]             

nxy = list(range(0, 3, 1))
# nxy = [0]
nn = nxy.__len__()

image_i = (dsa[nxy, ...]/(4096/256)).astype(np.uint8)
image_c = dsa_amp_pl[0, nxy, ...]
image_a = dsa_arc_pl[0, nxy, ...]
print(image_c.mean(axis=(1,2)))

mmclamm = 750
## V mm
cc_mask_stddev = 3
cc_mask_step = 3
cc_mask_threshold = .5
cc_score_cutoff = .5
cc_radii_cutoff = 5.1
cc_range = (3, 16, cc_mask_step)
 
mm_clamp = (mmclamm, mmclamm+1)
image_c = np.clip((image_c-mm_clamp[0])/(mm_clamp[1]-mm_clamp[0]), 0, 1)

image_cd = xd(image_c).astype(proc_dtype)
image_ad = xd(image_a)

ccrrl = list([1, 2]) + list(range(*cc_range))
ccrrl = sorted(ccrrl)
assert any(crx < cc_radii_cutoff for crx in ccrrl)

nd_2d2z = (*AXNEW, *AXALL, *AXALL)
if True:
    pad_cc_radius = ceil(max(ccrrl) + (cc_mask_stddev*2))
    pad_ms_radius = ceil(max(ccrrl)) #//2
    pad_ms_1d  = (pad_ms_radius,) *2
    pad = (*AXALL, slice(pad_cc_radius, image_c.shape[1]-pad_cc_radius), slice(pad_cc_radius, image_c.shape[2]-pad_cc_radius))

    sh_padded = image_c.shape
    sh_unpadd = tuple(na(sh_padded) - na([0,1,1])*2*pad_cc_radius)
    sh_mspadd = tuple(na(sh_unpadd) + na([0,1,1])*2*pad_ms_radius)

    pad_ms_mxa = xp.mgrid[(slice(-pad_ms_radius, pad_ms_radius+1),)*2].astype(index_dtype)
    pad_ms_amp = xp.sqrt((pad_ms_mxa.astype(radii_dtype)**2).sum(axis=0))
    pad_ms_mask = (pad_ms_amp <= pad_ms_radius) * (pad_ms_amp > 0)
    iter_adjmax_mask = tuple((pad_ms_amp[nd_2d2z][kernel_ixt], patch_sxt) 
        for kernel_ixt, patch_sxt in kernel_apply(sh_mspadd, pad_ms_mask[(*AXNEW,)])[-1])

    pad_cc_mxa = xp.mgrid[(slice(-pad_cc_radius, pad_cc_radius+1),)*2].astype(index_dtype)
    pad_cc_amp = xp.sqrt((pad_cc_mxa.astype(radii_dtype)**2).sum(axis=0))
    pad_cc_arc = xp.arctan2(pad_cc_mxa[1], pad_cc_mxa[0])

circ_gen = []
for ixcr, ccrr in enumerate(ccrrl):
    cc_mask_stddev = cc_mask_stddev 
    ccr_rim = ccrr
    cc_divs = ccr_rim*2

    cc_mask = xp.exp(-((pad_cc_amp-ccr_rim)/cc_mask_stddev)**2)
    cc_mask[cc_mask < cc_mask_threshold] = 0
    cc_mask /= (cc_divs)

    cc_mask_d = cc_mask[nd_2d2z]

    cc_arc_discr = (2*xp.pi)/cc_divs
    cc_mask_arc_discr = xp.floor((pad_cc_arc[nd_2d2z]/cc_arc_discr))
    # cc_mask_arc = ((cc_mask_arc_discr+.5)*cc_arc_discr)

    iter_ccmask_rad = tuple((
            (cc_discr+.5)*cc_arc_discr,
            tuple((
                tuple(kernel_ixt),
                tuple(slice(kix, asx + kix) for kix, asx in zip(kernel_ixt, sh_unpadd)),
            ) for kernel_ixt in xa(xp.where((cc_mask_d!=0)*(cc_mask_arc_discr==cc_discr))).astype(index_dtype).T))
        for cc_discr in range(-cc_divs//2, cc_divs//2+1))
    circ_gen.append((cc_mask_d, ccr_rim, iter_ccmask_rad))
print("INIT_DONE")

circ_score = xp.zeros(sh_unpadd, dtype=xp.float16)
circ_scsum = xp.zeros(sh_unpadd, dtype=xp.float16)
circ_radii = xp.zeros(sh_unpadd, dtype=xp.float16)

for (cc_mask_d, ccr_rim, iter_ccmask_rad,) in circ_gen:
    pdop_ccmask_rad = xp.zeros(sh_unpadd, dtype=proc_dtype)
    for kernel_arc, iter_ccmask in iter_ccmask_rad:
        pdop_ccmask_ri = xp.zeros(sh_unpadd, dtype=proc_dtype)
        image_adr = xp.cos(image_ad-kernel_arc)**8
        for kernel_ixt, patch_sxt, in iter_ccmask: 
            pccm = image_cd[patch_sxt] * cc_mask_d[kernel_ixt] * image_adr[patch_sxt]
            pdop_ccmask_ri = xp.maximum(pdop_ccmask_ri, pccm)
        pdop_ccmask_rad += pdop_ccmask_ri

    circ_score_recalc = pdop_ccmask_rad 
    ccovsc_bxa = (circ_score < circ_score_recalc)
    circ_radii = np.where(ccovsc_bxa, ccr_rim,              circ_radii)
    circ_score = np.where(ccovsc_bxa, circ_score_recalc,    circ_score)
    circ_scsum += pdop_ccmask_rad#

mspad = ((0,)*2, pad_ms_1d, pad_ms_1d,)
circ_scsum_pd = xp.pad(circ_score, mspad, constant_values=0)
circ_score_pd = xp.pad(circ_score, mspad, constant_values=0)
circ_radii_pd = xp.pad(circ_radii, mspad, constant_values=0)

ccscore_redux =  (cc_radii_cutoff <= circ_radii) * (cc_score_cutoff <= circ_score)
ccscore_redux_pd = xp.pad(ccscore_redux, mspad, constant_values=0)

ms_radii = circ_radii * ccscore_redux
for kernel_rad, patch_sxt in iter_adjmax_mask: 
    ms_condi = (True
        * ccscore_redux_pd[patch_sxt] 
        * (circ_radii_pd[patch_sxt] + kernel_rad > circ_radii)
    )
    ms_radii += ms_condi * (1/(2*xp.pi*kernel_rad))
ms_radii_pd = xp.pad(ms_radii, mspad, constant_values=0)

circ_filteredout = ms_radii * ccscore_redux
for kernel_rad, patch_sxt in iter_adjmax_mask: 
    circ_filteredout *= (False
        + (ms_radii_pd[patch_sxt] <= ms_radii) # ms circ bigger
        + (circ_radii_pd[patch_sxt] <= circ_radii*0 + kernel_rad) #ms circ or lesser and out of range
    )

circ_radii_ = circ_radii
ccscore_redux = circ_filteredout
circ_radii_ = ms_radii


print("CALC_DONE")

#####
c_where = xp.where(ccscore_redux)
c_shift = c_where[0].shape[0]
circ_where_cc_ = xa(c_where).T
circ_score_cc_ = circ_score[c_where]
circ_radii_cc_ = circ_radii_[c_where]

axx = (nn, 4)
# axx = (2, 4)
fig, ax = plt.subplots(*axx, figsize=4*na(axx[::-1]))
if ax.shape.__len__()==1: ax = na([ax])
fig.tight_layout()


#####
"""
ax[1, 2].imshow(xh((ms_radii-circ_radii)[0]))
ax[1, 3].imshow(xh((ms_radii)[0]))

ax[1, 0].imshow(xh(circ_score[0]))
# ax[1, 2].imshow(xh((circ_score * (circ_radii > cc_radii_cutoff))[0]))
ax[1, 1].imshow(xh(circ_radii[0]))
# ax[1, 3].imshow(xh((circ_radii * (circ_score > cc_score_cutoff))[0]))

withcircles = image_i[(slice(None), *pad[1:])].copy()
withdots = image_i[(slice(None), *pad[1:])].copy()
for nnx in range(1):
    dicsriminator = circ_where_cc_[:, 0] == nnx
    circ_where_cc = circ_where_cc_[dicsriminator, 1:]
    circ_radii_cc = circ_radii_cc_[dicsriminator]
    circ_score_cc = circ_score_cc_[dicsriminator]
    print("mean score hist:", np.histogram(xh(circ_score_cc)))

    for ci, cw, cr in zip(circ_score_cc, circ_where_cc, circ_radii_cc): 
        cv2.circle(withcircles[nnx], xh(cw[::-1]), int(cr), color=4000, thickness=1)
        withdots[(nnx, *xh(cw))] = 4000

    ax[nnx, 0].imshow(withcircles[nnx])
    ax[nnx, 1].imshow(withdots[nnx])
    ax[nnx, 2].imshow(image_c[nnx][pad[1:]])
    # ax[nnx, 3].imshow(xh(pdop_adjmax_cent[nnx][pad[1:]]))
    ax[nnx, 3].imshow(image_a[nnx][pad[1:]])
    # ax[2, 3].imshow(image_a[nnx][pad[1:]])
"""
#####
withcircles = image_i[(slice(None), *pad[1:])].copy()
withdots = image_i[(slice(None), *pad[1:])].copy()
for nnx, _ in enumerate(nxy):
    # if nnx == 0: continue
    dicsriminator = circ_where_cc_[:, 0] == nnx
    print(nnx, xh(dicsriminator).sum())
    circ_where_cc = circ_where_cc_[dicsriminator, 1:]
    circ_radii_cc = circ_radii_cc_[dicsriminator]
    circ_score_cc = circ_score_cc_[dicsriminator]
    # print("mean score hist:", np.histogram(xh(circ_score_cc)))

    for ci, cw, cr in zip(circ_score_cc, circ_where_cc, circ_radii_cc): 
        cv2.circle(withcircles[nnx], xh(cw[::-1]), int(cr), color=4000, thickness=1)
        withdots[(nnx, *xh(cw))] = 4000

    ax[nnx, 0].imshow(withcircles[nnx])
    ax[nnx, 1].imshow(withdots[nnx])
    ax[nnx, 2].imshow(image_c[nnx][pad[1:]])
    # ax[nnx, 3].imshow(xh(pdop_adjmax_cent[nnx][pad[1:]]))
    ax[nnx, 3].imshow(image_a[nnx][pad[1:]])

###
def mouse_callback(event):
    if int(event.button) ==3: print("marked: ", end="")
    on_canvas = event.inaxes is not None
    xy = (int(event.xdata), int(event.ydata)) if on_canvas else (None, None)
    print(xy, circ_scsum[0][xy[::-1]], circ_score[0][xy[::-1]], circ_radii[0][xy[::-1]]) #, eeee[xy[::-1]]) #, pdop_ccmask[xy[::-1]])
    if int(event.button) ==3:
        for axi in ax.flatten():
            axi.scatter(*xy, c='r', marker='+')
cid = fig.canvas.mpl_connect('button_press_event', mouse_callback)


[583.  572.5 579.5]
INIT_DONE
CALC_DONE
0 46
1 57
2 53


In [ ]:
## DATA READ > DYNAMIC

%matplotlib inline
xd, xh, xa, xp              = xm[cp]             
yd, yh, ya, yp              = xm[cp]

try: print(xp.zeros(0).device)
except BaseException: print("CPU!")

bnd2exc                 = na([[-1], [ 1]])

ds_ll = ds_l[:]
dslice_buffer           = (ds for ds in ds_ll)
def dslice_buffer2(n=9999):
    _ds_l = ds_ll
    splitn = np.random.randint(_ds_l.__len__()//32) *32
    print("Random dslice no = {}".format(splitn))
    ds_l2 = [*_ds_l[splitn:], *_ds_l[splitn:]]
    for ds in ds_l2[:n]:
        yield ds

corners_2d              = na(np.meshgrid(*(((0, 1),) *2))).reshape((2, -1)).T                   # [[0,0],[0,1],[1,0],[1,1]]
corners_3d              = na(np.meshgrid(*(((0, 1),) *3))).reshape((3, -1)).T                   # aa/3d
d_corners_2d            = xd(corners_2d)
d_corners_3d            = xd(corners_3d)

euler_to_matrix_rad = lambda roll_rad, pitch_rad, yaw_rad: np.array([
    [np.cos(yaw_rad) * np.cos(pitch_rad), -np.sin(yaw_rad) * np.cos(roll_rad) + np.cos(yaw_rad) * np.sin(pitch_rad) * np.sin(roll_rad), np.sin(yaw_rad) * np.sin(roll_rad) + np.cos(yaw_rad) * np.sin(pitch_rad) * np.cos(roll_rad)],
    [np.sin(yaw_rad) * np.cos(pitch_rad), np.cos(yaw_rad) * np.cos(roll_rad) + np.sin(yaw_rad) * np.sin(pitch_rad) * np.sin(roll_rad), -np.cos(yaw_rad) * np.sin(roll_rad) + np.sin(yaw_rad) * np.sin(pitch_rad) * np.cos(roll_rad)],
    [-np.sin(pitch_rad), np.cos(pitch_rad) * np.sin(roll_rad), np.cos(pitch_rad) * np.cos(roll_rad)]
])
euler_to_matrix_deg = lambda roll, pitch, yaw: np.array([
    [np.cos(np.radians(yaw)) * np.cos(np.radians(pitch)), -np.sin(np.radians(yaw)) * np.cos(np.radians(roll)) + np.cos(np.radians(yaw)) * np.sin(np.radians(pitch)) * np.sin(np.radians(roll)), np.sin(np.radians(yaw)) * np.sin(np.radians(roll)) + np.cos(np.radians(yaw)) * np.sin(np.radians(pitch)) * np.cos(np.radians(roll))],
    [np.sin(np.radians(yaw)) * np.cos(np.radians(pitch)), np.cos(np.radians(yaw)) * np.cos(np.radians(roll)) + np.sin(np.radians(yaw)) * np.sin(np.radians(pitch)) * np.sin(np.radians(roll)), -np.cos(np.radians(yaw)) * np.sin(np.radians(roll)) + np.sin(np.radians(yaw)) * np.sin(np.radians(pitch)) * np.cos(np.radians(roll))],
    [-np.sin(np.radians(pitch)), np.cos(np.radians(pitch)) * np.sin(np.radians(roll)), np.cos(np.radians(pitch)) * np.cos(np.radians(roll))]
])

def timeit(gene=None, desc=None):
    gene = gene if gene else (None,)
    for x in gene:
        t = datetime.datetime.now()
        yield x
        if desc:
            print(desc, datetime.datetime.now() -t)

""" spatial types:
# type          applies to      subtypes    dimensional ref
# spatial       db              sp, res     s
# chunk         dc                          c
# cgrid         db                          g           -dimissed
# planar        ds, dp          sh          p
# quantified                    sh, ix      -
"""

ndim                    = 3
dblock_resolution_mm    = (1,)     *ndim                                                        ## [mm] s{x,y,z}
dblock_chunk_shape      = (8,)     *ndim                                                        ## [ix] s{x,y,z}   

proc_dtype              = np.int16
meas_dtype              = np.float16
index_dtype             = np.int16

interp_max_dist         = 2                                                                     ## [mm]
continous               = True                                                                  # interpolate beetwen consecutive slices
adjust                  = True                                                                  # ... dblock space

## dblock > volumetric parametrization
if True:
    db_res                  = na(dblock_resolution_mm,                          dtype=meas_dtype)
    dc_sh                   = na(dblock_chunk_shape)                                                            
    dc_sp                   = db_res * dc_sh                                                                    # = spatial volume

    ddc_sp                  = xa(dc_sp)         
    ddc_sh                  = xa(dc_sh)
    ddc_mpa                 = xp.mgrid[tuple(slice(0, spx, res) for spx, res    in zip(dc_sp, db_res))  ]
    ddc_mxa                 = xp.mgrid[tuple(slice(0, shx)      for shx         in dc_sh)               ]

## dslice > init for CONSTANT parameters and dblock init      
# gene                    = (ds for ds in dslice_buffer2(200))               
gene                    = (dslice_buffer2(1))    ## DISABLE EXECUTION
if True:
    _dslice                 = gene.__next__()           
    ds_res                  = na(_dslice.PixelSpacing,                          dtype=meas_dtype)               ## [mm] p{x,y}
    dds_res                 = xd(ds_res,                                        dtype=meas_dtype)
    ds_sh                   = na(_dslice.pixel_array.shape) #//2 # dg            
    ds_sp                   = ds_sh * ds_res            

    _ds_thickness           = na(_dslice.SliceThickness,                        dtype=meas_dtype)               ## [mm] sz?
    ds2b_res                = (1/db_res) * ds_res[:, None]                                                      ## [ix/ix]  {p{x,y} / s{x,y,z}}
    ds_corners_sp           = corners_2d * ds_sp            

## dblock > init            
_db_init_pos            = na(_dslice.ImagePositionPatient,                  dtype=meas_dtype) #- interp_max_dist ## [mm] s{x,y,z}
_db_init_shape          = (0,) *ndim            

dblock_rev              = np.diag((1,) *ndim, 0)                                                            # SI:DBREV
db_spb                  = na((_db_init_pos,) *2)                                                            # {{l,u} * {x,y,z}}; spatial boundaries (upper one is cache by db_sh)
dblock                  = np.zeros(_db_init_shape,                          dtype=proc_dtype)               ## {s{x,y,z}}

assert np.all(dblock_rev == np.diag((1,) *ndim, 0))                                                         # simplicity issue SI:DBERV: using basic base to reduce (& implementing) adjustment revs
db_utrack               = np.zeros(_db_init_shape,                          dtype=meas_dtype)               # tracking updates timing via .AcquisitionTime # todo
db_sh                   = None

## save previous slice vals for interpolation update (atm allowing losing init dslice)  # todo: fulfill saving specified vals in loop
force_adjust            = True
interpolable            = False                                                                 # flag for dp info completeness

# for dslice in gene:
for dslice in timeit(gene, "cycle"):
    ## calc     > dslice properties
    if True:
        ds_array            = na(dslice.pixel_array,                        dtype=proc_dtype)

        ## calc dslice properties  @(abs)
        _ds2a_pos           = na(dslice.ImagePositionPatient,               dtype=meas_dtype)                   ## [mm] s{x|y|z}
        _ds2a_p2s           = na(dslice.ImageOrientationPatient,            dtype=meas_dtype).reshape((2, 3))   ## [1]  p{x2s{x|y|z} | y2s{x|y|z}}
        _ds_pos_a0          = _ds2a_pos                                     @ dblock_rev.T
        ds_p2s              = _ds2a_p2s                                     @ dblock_rev.T
        # ds_p2s = ds_p2s @ euler_to_matrix_deg(0,45,45) # dg
        ds_normal           = np.cross(*ds_p2s)                                                     # always unit vec (p2s components always stereometrically perpendicular)
        ds_rev              = na((*ds_p2s, ds_normal))                                              # rotation matrix
        ds_midb             = np.abs(interp_max_dist * ds_normal) * bnd2exc                         # maximal interpolation distance boundaries

        ## calc dslice properties   > calc excess, adjust dblock  @(db_pos, db_rev)
        _ds_spb_a0          = np.matmul(ds_corners_sp, ds_p2s) + _ds_pos_a0
        _ds_spb_abs         = na((  _ds_spb_a0.min(axis=0), 
                                    _ds_spb_a0.max(axis=0),))
        _ds_spb_ext_abs     = _ds_spb_abs + ds_midb                                                                        

        ds_sp_exc           = np.maximum((_ds_spb_ext_abs - db_spb) *bnd2exc, 0)                    ## {{l|r}, {x|y|z}} spatial excess over dblock; SI: is extended for roi crop for interpolation (to dimiss minmaxing for dblock boundaries)
        if (adjust and np.any(ds_sp_exc > 0)) or force_adjust:                                      # if dslice excesses beyond dblock
            print("dblock padded")
            _ds2c_shexc     = np.ceil(ds_sp_exc / dc_sp)                    .astype(index_dtype)    # pad with chunk shape
            _db_spbexc      = _ds2c_shexc * dc_sp
            _db_shbexc      = _ds2c_shexc * dc_sh

            db_spb          = db_spb + (_db_spbexc * bnd2exc)
            dblock          = np.pad(dblock,   _db_shbexc.T, "constant", constant_values=0)
            force_adjust    = False

        ds_spb              = _ds_spb_ext_abs   - db_spb[0]                                         # shift dslice position (after adjustment); operating in common spatial dimensions
        ds_pos              = _ds_pos_a0        - db_spb[0]

        ## calc dslice properties   > calc affected chunks region boundaries
        ds_scb              = ((ds_spb / dc_sp) + na([[0],[1]]))            .astype(index_dtype)
        ds_sx_rel           = ds_scb[1] - ds_scb[0]

        ## calc dslice properties   > to device
        dds_pos             = xd(ds_pos)
        dds_array           = xd(ds_array)
        dds_p2s             = xd(ds_p2s)
        dds_normal          = xd(ds_normal)
        dds_midb            = xd(ds_midb)
        dds_scb             = xd(ds_scb)

    ## qualify  > dslice    > chunks affected  @(ds_sc_pos, db_rev)
    if True:
        _dds_scb_mvpae      = xp.moveaxis(
                                xp.mgrid[tuple(
                                    slice(      spl, spu + res,     res) 
                                    for         spl, spu,           res in 
                                    np.vstack(  (ds_scb * dc_sp,    dc_sp)).T)
                                ].astype(meas_dtype), 
                                0, -1) - dds_pos  # device-data-slice_shape:chunk-boundaries_meshgrid-moveaxed-spatial-array-extended = spatial corners of chunks; intercept from eq: a*(x-x0) + b*(y-y0) + c(z-z0) = 0, yet adjusted to db[_sp_ogrid]
        _dds_scb_mxa_cr     = tuple(d_corners_3d.T[(..., *((None,)*ndim))] 
                                + xp.mgrid[tuple(slice(scb) for scb in ds_sx_rel)][:, None])        # {t:ndim, c{x,y,z}} => {t:ndim, cr=2**ndim, c{x,y,z}} - corners take
        
        _dds_scb_mpae_mddist= xa(tuple(xp.sum((_dds_scb_mvpae + dds_midbx) * dds_normal, axis=-1) for dds_midbx in dds_midb))
        _ddcc_planar        = xp.logical_or(
                                xp.mod((_dds_scb_mpae_mddist[0][_dds_scb_mxa_cr] > 0).sum(axis=0), d_corners_3d.__len__()),
                                xp.mod((_dds_scb_mpae_mddist[1][_dds_scb_mxa_cr] > 0).sum(axis=0), d_corners_3d.__len__()),)   # verify which chunk corners directed_distance values different signs = dslice+-imd*normal passes through them; assert (interp_max_dist << dc_sp <=> (ds passes chunk == chunk between dslice+-imd*normal) bcof this
        """
        t = datetime.datetime.now()
        _dds_scb_mxa_rel_cr = tuple(_dds_scb_mxa_rel[..., None] + d_corners_3d.T[(slice(None), *((None,) *ndim), slice(None))])
        ddcc_planar_         = xp.logical_or(
                                xp.mod((dds_scb_mpae_mddist[0][_dds_scb_mxa_rel_cr][..., None] > 0).sum(axis=-2), 2**ndim).squeeze(axis=-1),
                                xp.mod((dds_scb_mpae_mddist[1][_dds_scb_mxa_rel_cr][..., None] > 0).sum(axis=-2), 2**ndim).squeeze(axis=-1),)   
        print((datetime.datetime.now() - t)) 
        # t = datetime.datetime.now()
        # ddcc_planar_u       = xp.zeros((dds_midb.__len__(), *ds_sx_rel),                    dtype=index_dtype)
        # for dcr in d_corners_3d:
        #     ddcc_planar_u   += dds_scb_mpae_mddist[(slice(None),) + tuple(slice(dcx, scb + dcx) for scb, dcx in zip(ds_sx_rel, dcr))] > 0
        # ddcc_planar          = xp.logical_or(*xp.mod(ddcc_planar_u, d_corners_3d.__len__())) # use only with interp_max_dist << dc_sh  # print(ha(ddcc_planar).sum(), "/", np.prod(ds_sx_rel))
        # print((datetime.datetime.now() - t))
        # print((ha(ddcc_planar == ddcc_planar_)).all())
        """
        _dds_scb_mpae_dsproj= (tuple(xp.sum((_dds_scb_mvpae) * dds_p2sx, axis=-1) for dds_p2sx in dds_p2s))
        _ddcc_proj          = xa((  
                                0 <= _dds_scb_mpae_dsproj[0][_dds_scb_mxa_cr], _dds_scb_mpae_dsproj[0][_dds_scb_mxa_cr] <= ds_sp[0]-1,
                                0 <= _dds_scb_mpae_dsproj[1][_dds_scb_mxa_cr], _dds_scb_mpae_dsproj[1][_dds_scb_mxa_cr] <= ds_sp[1]-1,
                            )).all(axis=1).all(axis=0)  # Performance issue rel2aa [dc_proj_chunked]; alt: # 
                            # )).any(axis=1).all(axis=0)
        dds_ac_mca          = xp.logical_and(_ddcc_planar, _ddcc_proj)                                # device data chunk condition

    ## qualify  > dinterp   > chunks affected @(db_pos, db_rev) #todo: batchify?
    if interpolable:  # for _ in timeit("calc chunks cond"): # 0.286
        _ddiac_spb          = xa((  xp.maximum(ddp_scb[0], dds_scb[0]),
                                    xp.minimum(ddp_scb[1], dds_scb[1])))
        _ddiac_mca          = xp.logical_and(
                                dds_ac_mca[tuple(slice(scl, scu) for scl, scu in (_ddiac_spb - dds_scb[0]).T)],
                                ddp_ac_mca[tuple(slice(scl, scu) for scl, scu in (_ddiac_spb - ddp_scb[0]).T)],)
        ddiac_l             = _ddiac_spb[0] + xa(xp.where(_ddiac_mca)).T
        """ batchified
        # for _ , ddiabc_l in batchify(ddiac_l, patch_shape=dc_sh, umem_pool=2**31, umem_poly=(*(0,)*ndim, 600), tv=None, xp=xp):
        # for ddiabc_l in (ddiac_l.T,):
            # ddiac_lmvpa         = ((xa(ddiabc_l).T * ddc_sp)[(..., *(None,) *ndim)] + ddc_mpa).swapaxes(1, -1) ##{l, c{x,y,z}, ndim}; 
            # ddiac_lmxa          = ((xa(ddiabc_l).T * ddc_sh)[(..., *(None,) *ndim)] + ddc_mxa).swapaxes(0, 1)  ##{ndim, l, c{x,y,z}}
        """

        ddiac_lmvpa         = np.moveaxis((ddiac_l * ddc_sp)[(..., *(None,) *ndim)] + ddc_mpa, 1, -1) ##{l, c{x,y,z}, ndim}; 
        ddiac_lmxa          = np.moveaxis((ddiac_l * ddc_sh)[(..., *(None,) *ndim)] + ddc_mxa, 0, 1)  ##{ndim, l, c{x,y,z}}
        print("ddiac_lmvpa", ddiac_lmvpa.shape)
    
    ## qualify  > dinterp   > voxels
    if interpolable:
        _ddiac_mpa_mddist   = xa(tuple(
                                xp.sum((ddiac_lmvpa - ddi_pos) * ddi_normal, axis=-1) 
                                for ddi_normal, ddi_pos in ((ddp_normal, ddp_pos), (dds_normal, dds_pos))))
        _ddiac_mpa_madist   = xp.abs(_ddiac_mpa_mddist)
        assert xp.dot(ddp_normal, dds_normal) > 0
        _ddiac_planar       = xp.logical_and(
                                _ddiac_mpa_mddist[0] * _ddiac_mpa_mddist[1] <= 0,
                                _ddiac_mpa_madist[0] + _ddiac_mpa_madist[1] <= interp_max_dist)
        _dds_mpa_dsproj     = xa(tuple(tuple(
                                xp.sum((ddiac_lmvpa - ddi_pos) * ddi_p2s_x, axis=-1) 
                                for ddi_p2s_x in ddi_p2s)
                                for ddi_p2s, ddi_pos in ((ddp_p2s, ddp_pos), (dds_p2s, dds_pos))))  ## {{dp|ds}, ds{x|y}, l, sp{x,y,z}}
        _ddiac_proj         = xp.ones(_ddiac_planar.shape, dtype=bool)  # Performance issue ad:[dc_proj_chunked]; alt: 
        # _ddiac_proj         = xa((0 <= _dds_mpa_dsproj[:, 0], _dds_mpa_dsproj[:, 0] <= ds_sp[0]-1, 0 <= _dds_mpa_dsproj[:, 1], _dds_mpa_dsproj[:, 1] <= ds_sp[1]-1,)).all(axis=(0, 1))  # SI: assuming ds_sp is constant
        diac_cond           = xp.logical_and(_ddiac_planar, _ddiac_proj)

    ## calc     > dinterp   > voxels
    if interpolable:
        vxval               = xp.zeros(diac_cond.shape, dtype=proc_dtype)
        for dsix in (0,1):
            _dsar           = (ddp_array, dds_array)[dsix]
            _dsdw           = _ddiac_mpa_madist[dsix] / _ddiac_mpa_madist.sum(axis=0)
            _dspj           = _dds_mpa_dsproj[dsix]
            _dspj_f         = (_dspj / dds_res[(..., *(None,) * (ndim+1))]) ## {ds{x|y}, l, sp{x,y,z}}
            _dspj_i         = _dspj_f.astype(index_dtype)
            _dspj_r         = _dspj_f - _dspj_i

            _dspj_i_cr      =        d_corners_2d.T[(..., *(None,)*(ndim+1))] + _dspj_i[:, None]
            _dspj_r_cr      = xp.abs(d_corners_2d.T[(..., *(None,)*(ndim+1))] - _dspj_r[:, None])

            _vxvala         = (_dsar[tuple(_dspj_i_cr)] * _dspj_r_cr.prod(axis=0)).sum(axis=0) * _dsdw
            vxval           += (_vxvala).astype(proc_dtype)

        dblock[tuple(xh(ddiac_lmxa))] = np.where(xh(diac_cond), xh(vxval), dblock[tuple(xh(ddiac_lmxa))])
        # dblock[tuple(xh(ddiac_lmxa))] = np.where(xh(diac_cond), i, dblock[tuple(xh(ddiac_lmxa))])

    ## cache    > previous dslice
    if True:
        interpolable        = True
        ddp_array           = dds_array
        ddp_ac_mca          = dds_ac_mca
        ddp_normal          = dds_normal
        ddp_p2s             = dds_p2s
        ddp_scb             = dds_scb
        ddp_pos             = dds_pos


